In [ ]:

import numpy as np
import pandas as pd
import scipy as sp
import scipy.cluster.hierarchy as hier
import scipy.spatial.distance as dist
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
gene = pd.read_excel('gene_expression_table4.xlsx')
#print(gene)
datamatrix=[]
del gene['Unnamed: 9'] # deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:14]
print(just_expression)
just_expression.as_matrix  # transformed dataframe with expression values to array
just_expression.shape
dist_matrix= dist.pdist(just_expression)
dist_square_matrix = dist.squareform(dist_matrix)
#print(dist_square_matrix)

In [ ]:
# Compute and plot dendrogram.
fig = pl.figure()
axdendro = fig.add_axes([0.09,0.1,0.2,0.8])
linkageMatrix = hier.linkage(dist_square_matrix, method='centroid') # created linkage matrix
dendrogram = hier.dendrogram(linkageMatrix, orientation='left') # produced dendrogram of the linkage matrix                             #Y = sch.linkage(D, method='centroid')
#Z = sch.dendrogram(Y, orientation='right')
axdendro.set_xticks([])
axdendro.set_yticks([])

# Plot distance matrix.
axmatrix = fig.add_axes([0.3,0.1,0.6,0.8])
index = dendrogram['leaves']
im = axmatrix.matshow(just_expression, aspect='auto', origin='lower')
axmatrix.set_xticks([])
axmatrix.set_yticks([])

# Plot colorbar.
axcolor = fig.add_axes([0.91,0.1,0.02,0.8])
pl.colorbar(im, cax=axcolor, label= 'log2 expression')

# Display and save figure.
fig.show()
fig.savefig('dendrogram.png')
fig.set_figheight(10)

In [ ]:
#print(just_expression)
BRCA1_plus= just_expression.iloc[:,0:6]
#print(BRCA1_plus.T)
BRCA1_minus= just_expression.iloc[:,7:12]
#print(BRCA1_minus.T)

BRCA1_plus_means=np.mean(BRCA1_plus.T)
#print(BRCA1_plus_means.shape)
BRCA1_minus_means=np.mean(BRCA1_minus.T)
#print(BRCA1_minus_means.shape)

concatenated_means= np.c_[BRCA1_plus_means,BRCA1_minus_means]
#print(concatenated_means)

expression_change_abs= np.abs(np.diff(concatenated_means))
#print(expression_change_abs.shape)

expression_change_raw= np.diff(concatenated_means)
#print(expression_change_raw)

r=np.append(expression_change_abs,expression_change_raw,1)
print(r)

In [ ]:
a=gene.iloc[0:373,0:3]
print(a.shape)
names=['mean','raw mean']
mean_pandas_abs= pd.DataFrame(r,columns=names)
print(mean_pandas.shape)

mean_append= np.append(a,mean_pandas,1)

names2= ['gene name', 'symbol','function','abs value mean difference','raw mean difference']
h=pd.DataFrame(means2_append,columns=names2)

sorted_means=h.sort(['abs value mean difference'],ascending=False)
print(sorted_means)




In [ ]:
 
fig= plt.figure()

#plt.subplot(211)
#plt.plot(np.arange(373),sorted_means.iloc[:,3],':')
fig.set_figheight(10)
fig.set_figwidth(20)
#plt.subplot(212)
#plt.plot(np.arange(373),sorted_means.iloc[:,4],'.')


f=np.unique(sorted_means.iloc[:,2])
print(f)

for i in np.unique(sorted_means.iloc[:,2]):
    plt.plot(np.arange(373),sorted_means.iloc[:,3],':' )


In [ ]:
u=sorted_means.groupby(['function'])
u.describe
